In [8]:
# 导入所需库
import ccxt
import pandas as pd
import datetime
import time
import os

In [10]:
def save_spot_candle_data_from_exchange(exchange, symbol, time_interval, start_time, path):
    """
    从交易所获取并保存指定日期和交易对的K线数据
    返回: (DataFrame, bool) - 数据框和是否为空数据的标志
    """
    df_list = []
    start_time_since = exchange.parse8601(start_time)
    end_time = pd.to_datetime(start_time) + datetime.timedelta(days=1)

    while True:
        try:
            # 获取K线数据
            klines = exchange.fetch_ohlcv(
                symbol=symbol, 
                timeframe=time_interval, 
                since=start_time_since
            )
            if not klines:
                break
                
            # 创建DataFrame，保持与官方API完全一致的处理方式
            df = pd.DataFrame(klines)
            # 只保留前6列（与官方API一致）
            if len(df.columns) > 6:
                df = df.iloc[:, :6]
            
            df_list.append(df)
            
            last_timestamp = df.iloc[-1][0]  # 第一列是时间戳
            start_time_since = int(last_timestamp) + 1
            
            if pd.to_datetime(last_timestamp, unit='ms') >= end_time or df.shape[0] <= 1:
                break
                
            time.sleep(0.5)
            
        except Exception as e:
            print(f"获取数据出错: {e}")
            time.sleep(3)
            continue

    if not df_list:
        return pd.DataFrame(), True  # 返回空DataFrame和空数据标志
    
    # 合并数据
    df = pd.concat(df_list, ignore_index=True)
    
    # 完全按照官方API的方式处理数据
    df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
    
    # 使用完全相同的数据类型转换方式
    df['datetime'] = df['datetime'].values.astype(dtype='datetime64[ms]')
    df['open'] = df['open'].values.astype(float)
    df['high'] = df['high'].values.astype(float)
    df['low'] = df['low'].values.astype(float)
    df['close'] = df['close'].values.astype(float)
    df['volume'] = df['volume'].values.astype(float)
    
    # 筛选当天数据
    target_date = pd.to_datetime(start_time).date()
    df = df[df['datetime'].dt.date == target_date]
    
    # 数据清理
    df = df.drop_duplicates(subset=['datetime'], keep='last')\
           .sort_values('datetime')\
           .reset_index(drop=True)
    
    # 保存数据
    if not df.empty:
        date_str = str(pd.to_datetime(start_time).date())
        date_path = os.path.join(path, date_str)
        os.makedirs(date_path, exist_ok=True)
        
        file_name = f"{date_str}_{symbol.replace('/', '')}_{time_interval}.csv"
        full_path = os.path.join(date_path, file_name)
        
        df.to_csv(full_path, index=False)
        print(f"数据已保存至: {full_path}")
    
    return df, False  # 返回数据和非空数据标志

In [15]:
# 设置数据获取参数
params = {
    # 数据获取配置
    'begin_date': '2024-01-01',
    'end_date': '2025-01-01',
    'time_intervals': ['1m'],  # 可选: ['1m', '5m', '15m', '1h', '4h', '1d']
    'use_all_usdt_pairs': False,  # 设置为True则获取所有USDT交易对，False则使用指定交易对
    'specific_symbols': ['KAS/USDT'],  # 当use_all_usdt_pairs为False时使用
    'base_path': r'\\znas\Main\spot',  # 指定数据保存的根目录
    
    # 代理配置
    'proxy': {
        'host': '127.0.0.1',
        'port': 2206
    },
    
    # 交易所基础配置
    'exchange_config': {
        'timeout': 5000,
        'enableRateLimit': False
    }
}


# 确保根目录存在
os.makedirs(params['base_path'], exist_ok=True)

In [16]:
# 生成日期列表
def generate_date_list(begin_date, end_date):
    date_list = []
    date = pd.to_datetime(begin_date)
    while date <= pd.to_datetime(end_date):
        date_list.append(str(date))
        date += datetime.timedelta(days=1)
    return date_list

date_list = generate_date_list(params['begin_date'], params['end_date'])
print(f"将获取从 {date_list[0]} 到 {date_list[-1]} 的数据")

将获取从 2024-01-01 00:00:00 到 2025-01-01 00:00:00 的数据


In [17]:
def initialize_exchange_and_markets(params):
    """
    初始化交易所并加载市场数据
    """
    # 添加代理配置到交易所配置中
    proxy_url = f"http://{params['proxy']['host']}:{params['proxy']['port']}"
    params['exchange_config']['proxies'] = {
        'http': proxy_url,
        'https': proxy_url
    }
    
    # 初始化交易所
    exchange = ccxt.binance(params['exchange_config'])
    
    # 加载市场数据
    while True:
        try:
            market = exchange.load_markets()
            print('市场数据加载成功')
            
            # 处理交易对
            market_df = pd.DataFrame(market).T
            all_symbols = list(market_df['symbol'])
            
            if params['use_all_usdt_pairs']:
                target_symbols = [symbol for symbol in all_symbols if symbol.endswith('/USDT')]
            else:
                target_symbols = params['specific_symbols']
                
            print(f"将获取以下交易对的数据：{target_symbols}")
            print(f"共 {len(target_symbols)} 个交易对")
            
            return exchange, target_symbols
            
        except Exception as e:
            print(f'加载市场数据失败: {e}')
            time.sleep(3)

# 初始化交易所并获取交易对
exchange, target_symbols = initialize_exchange_and_markets(params)

市场数据加载成功
将获取以下交易对的数据：['KAS/USDT']
共 1 个交易对


In [18]:

# 在主循环之前，添加预扫描函数
def scan_existing_files(base_path):
    """预扫描已存在的文件"""
    existing_files = set()
    for root, _, files in os.walk(base_path):
        for file in files:
            if file.endswith('.csv'):
                existing_files.add(file)
    print(f"已扫描到 {len(existing_files)} 个现有文件")
    return existing_files

# 获取现有文件列表
existing_files = scan_existing_files(params['base_path'])

# 添加统计函数
def analyze_download_status(target_symbols, existing_files, date_list, time_intervals):
    """分析每个交易对的下载情况"""
    stats = {}
    for symbol in target_symbols:
        stats[symbol] = {
            'total_expected': len(date_list) * len(time_intervals),
            'downloaded': 0,
            'missing_dates': []
        }
        
        for start_time in date_list:
            for time_interval in time_intervals:
                date_str = str(pd.to_datetime(start_time).date())
                file_name = f"{date_str}_{symbol.replace('/', '')}_{time_interval}.csv"
                if file_name in existing_files:
                    stats[symbol]['downloaded'] += 1
                else:
                    stats[symbol]['missing_dates'].append(f"{date_str}_{time_interval}")

    # 打印统计信息
    print("\n下载统计信息:")
    incomplete_symbols = []
    for symbol, data in stats.items():
        completion_rate = (data['downloaded'] / data['total_expected']) * 100
        print(f"{symbol}: 完成率 {completion_rate:.2f}% ({data['downloaded']}/{data['total_expected']})")
        if data['downloaded'] < data['total_expected']:
            incomplete_symbols.append(symbol)
            if len(data['missing_dates']) <= 10:  # 只显示前10个缺失日期
                print(f"  缺失数据: {data['missing_dates'][:10]}")
            else:
                print(f"  缺失数据过多，共{len(data['missing_dates'])}个日期")

    print(f"\n未完全下载的交易对数量: {len(incomplete_symbols)}/{len(target_symbols)}")
    return incomplete_symbols

# 在主循环结束后添加分析
print("\n开始分析下载情况...")
existing_files = scan_existing_files(params['base_path'])  # 重新扫描一次
incomplete_symbols = analyze_download_status(
    target_symbols, 
    existing_files, 
    date_list, 
    params['time_intervals']
)

# 如果需要，可以只针对未完成的交易对重新下载
if incomplete_symbols:
    print("\n是否要重新下载未完成的交易对？(y/n)")
    if input().lower() == 'y':
        # 更新target_symbols为未完成的交易对
        target_symbols = incomplete_symbols
        
# 主循环部分
error_list = []
empty_data_count = {}  # 用于记录每个交易对的连续空数据天数

# 反转日期列表，从最新日期开始获取
date_list.reverse()

for symbol in target_symbols:
    empty_data_count[symbol] = 0
    
    for start_time in date_list:
        if empty_data_count[symbol] >= 3:
            print(f'{symbol} 连续 {empty_data_count[symbol]} 天无数据，跳转到下一个交易对')
            break
            
        for time_interval in params['time_intervals']:
            try:
                # 直接检查文件名是否在集合中
                date_str = str(pd.to_datetime(start_time).date())
                file_name = f"{date_str}_{symbol.replace('/', '')}_{time_interval}.csv"
                
                if file_name in existing_files:
                    print(f"文件 {file_name} 已存在，跳过下载")
                    empty_data_count[symbol] = 0  # 重置计数器
                    continue
                
                # 如果文件不存在，则下载数据
                print(f'正在获取 {exchange.id} {symbol} {time_interval} {start_time} 的数据')
                df, is_empty = save_spot_candle_data_from_exchange(
                    exchange=exchange,
                    symbol=symbol,
                    time_interval=time_interval,
                    start_time=f'{start_time} 00:00:00',
                    path=params['base_path']
                )
                
                if is_empty or (isinstance(df, pd.DataFrame) and df.empty):
                    empty_data_count[symbol] += 1
                    print(f'{symbol} 连续 {empty_data_count[symbol]} 天无数据')
                    if empty_data_count[symbol] >= 3:
                        print(f'达到连续空数据阈值，跳转到下一个交易对')
                        break
                else:
                    empty_data_count[symbol] = 0
                    print(f'成功下载并保存 {symbol} 在 {start_time} 的数据，数据形状: {df.shape}')
                
            except Exception as e:
                error_msg = f'{exchange.id}_{symbol}_{time_interval}_{start_time}'
                print(f'获取数据失败: {error_msg}, 错误: {e}')
                error_list.append(error_msg)
                
            time.sleep(0.5)
            
        if empty_data_count[symbol] >= 3:
            break

已扫描到 0 个现有文件

开始分析下载情况...
已扫描到 0 个现有文件

下载统计信息:
KAS/USDT: 完成率 0.00% (0/367)
  缺失数据过多，共367个日期

未完全下载的交易对数量: 1/1

是否要重新下载未完成的交易对？(y/n)
正在获取 binance KAS/USDT 1m 2025-01-01 00:00:00 的数据
数据已保存至: \\znas\Main\future\2025-01-01\2025-01-01_KASUSDT_1m.csv
成功下载并保存 KAS/USDT 在 2025-01-01 00:00:00 的数据，数据形状: (1440, 6)
正在获取 binance KAS/USDT 1m 2024-12-31 00:00:00 的数据
数据已保存至: \\znas\Main\future\2024-12-31\2024-12-31_KASUSDT_1m.csv
成功下载并保存 KAS/USDT 在 2024-12-31 00:00:00 的数据，数据形状: (1440, 6)
正在获取 binance KAS/USDT 1m 2024-12-30 00:00:00 的数据
数据已保存至: \\znas\Main\future\2024-12-30\2024-12-30_KASUSDT_1m.csv
成功下载并保存 KAS/USDT 在 2024-12-30 00:00:00 的数据，数据形状: (1440, 6)
正在获取 binance KAS/USDT 1m 2024-12-29 00:00:00 的数据
数据已保存至: \\znas\Main\future\2024-12-29\2024-12-29_KASUSDT_1m.csv
成功下载并保存 KAS/USDT 在 2024-12-29 00:00:00 的数据，数据形状: (1440, 6)
正在获取 binance KAS/USDT 1m 2024-12-28 00:00:00 的数据
数据已保存至: \\znas\Main\future\2024-12-28\2024-12-28_KASUSDT_1m.csv
成功下载并保存 KAS/USDT 在 2024-12-28 00:00:00 的数据，数据形状: (1440, 6)
正在获取 bi

: 

In [27]:
# 显示执行结果
print('数据获取完成')
if error_list:
    print('\n获取失败的数据:')
    for error in error_list:
        print(error)
else:
    print('\n所有数据获取成功！')

数据获取完成

所有数据获取成功！
